In [1]:
import torch
print(torch.__version__)

2.1.0


In [2]:
import cv2
import numpy as np
from mtcnn import MTCNN
# from math import atan2, degrees
from deep_sort_realtime.deepsort_tracker import DeepSort
import face_recognition

In [3]:
# Initialize DeepSORT tracker
tracker = DeepSort(max_age=5)

In [4]:
# Load pre-trained MTCNN model for face detection
mtcnn = MTCNN()

In [5]:
# Open video file
video_path = "Class Room Entrance.mp4"
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

In [6]:
# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Use "mp4v" for mp4 format
output_path = "output_video.mp4"
out = cv2.VideoWriter(output_path, fourcc, 20.0, (frame_width, frame_height))

In [7]:
frame_id = 0
skip_frames = 2  # Skip the frames

In [8]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1

    # Skip frames until reaching the nth frame
    if frame_id <= skip_frames:
        continue

    # Detect faces using MTCNN
    faces = mtcnn.detect_faces(frame)
    bbs = [(face['box'], face['confidence'], face['keypoints']) for face in faces]

    # Update tracker with the detected faces
    tracks = tracker.update_tracks(bbs, frame=frame)

    # Draw bounding boxes and IDs on the frame
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()

        # Draw bounding box
        cv2.rectangle(frame, (int(ltrb[0]), int(ltrb[1])), (int(ltrb[2]), int(ltrb[3])), (0, 255, 0), 2)

        # Get facial landmarks using face_recognition library
        face_locations = [(int(ltrb[1]), int(ltrb[2]), int(ltrb[3]), int(ltrb[0]))]  # (top, right, bottom, left)
        landmarks = face_recognition.face_landmarks(frame, face_locations)

        if landmarks:
            # Extract and save the face image
            face_image = frame[int(ltrb[1]):int(ltrb[3]), int(ltrb[0]):int(ltrb[2])]

            # Check if the face image is not empty before saving
            if not face_image.size == 0:
                save_path = f"faces/{track_id}_frame{frame_id}.png"
                cv2.imwrite(save_path, face_image)

                # Draw label
                label = f"ID: {track_id}"
                cv2.putText(frame, label, (int(ltrb[0]), int(ltrb[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Write the frame with bounding boxes to the output video
    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 45ms/step
